In [5]:
import warnings
warnings.filterwarnings("ignore")

from time import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pandas as pd

from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns

from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_predict
pd.set_option('display.max_rows', None)
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_predict

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,RNN,LSTM,Activation,Dropout
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers

#para converter o modelo do tensorflow ara scikitlearn
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [6]:
df_crisis_boe = pd.read_excel('/content/sample_data/Bank_ready.xlsx')

df_crisis_mean = pd.read_excel('/content/sample_data/Mean_ready.xlsx') 

df_crisis_median = pd.read_excel('/content/sample_data/Median_ready.xlsx') 

df_crisis_mostf = pd.read_excel('/content/sample_data/Most_f_ready.xlsx')  

In [7]:
def datapreparation(df_crisis):

  dfx_crisis = df_crisis.drop(['year','iso'],axis=1) #retirar atributos sem relação a crises
  
  dfx_crisis = dfx_crisis.drop(['crisis'],axis=1) #retirar variavel binaria indicadora de crises

  dfy_crisis = df_crisis['crisis'].copy() 
  dfx_crisis.shape, dfy_crisis.shape
  
  dfx_crisis_resampled, dfy_crisis_resampled = SMOTE().fit_resample(dfx_crisis, dfy_crisis)
  dfx_crisis_resampled,dfy_crisis_resampled = shuffle(dfx_crisis_resampled,dfy_crisis_resampled)

  dfx_crisis_resampled = pd.DataFrame(dfx_crisis_resampled)
  dfy_crisis_resampled = pd.DataFrame(dfy_crisis_resampled)
  dfx_crisis_resampled.shape, dfy_crisis_resampled.shape

  dfx_crisis_resampled = dfx_crisis_resampled.to_numpy().reshape(-1, 1, 12)
  dfy_crisis_resampled = dfy_crisis_resampled.to_numpy().astype('float32').reshape((-1,1))

  return dfx_crisis_resampled, dfy_crisis_resampled

In [8]:
dfx_boe, dfy_boe = datapreparation(df_crisis_boe)
dfx_mean, dfy_mean = datapreparation(df_crisis_mean)
dfx_median, dfy_median = datapreparation(df_crisis_median)
dfx_mostf, dfy_mostf = datapreparation(df_crisis_mostf)

In [9]:
# Split the dataset in two equal parts
def spliter(dfx,dfy):
  X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.75, random_state=0)
  return X_train, X_test, y_train, y_test

In [10]:
# Set the parameters by cross-validation
tuned_parameters = [{'epochs': [10,20,30,40,45,50,55,60,65],
                     'batch_size': [10,13,15,17,20,25,30,40,50,60,70,80,90,100],
                     'verbose':[0]},
                    ]
scores = ['precision', 'recall']


In [11]:
# Na linha do modelo Hidden_nodes: 5,10,15
# model.add(LSTM(Hidden_nodes,return_sequences=True))

# Nalinha do modelo optimizer: 'sgd', 'adam' e 'rmsprop'
# model.compile(optimizer ,loss='binary_crossentropy',metrics=['accuracy'])

# Nalinha do modelo loss: 'binary_crossentropy' para categorização binária
# model.compile(optimizer ,loss,metrics=['accuracy'])

In [12]:
def create_model():
  model = Sequential()
  model.add(LSTM(15,return_sequences=True))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(LSTM(3))
  model.add(Activation('relu'))
  model.add(Dense(1))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  model.build(dfx_boe.shape)
  # print(model.summary())
  return model

def create_model():
  model = Sequential()
  # model.add(layers.Embedding(input_dim=2045, output_dim=dfx_boe.shape[1]))
  model.add(LSTM(25, return_sequences=True))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(LSTM(3))
  model.add(Activation('relu'))
  model.add(Dense(1))
  model.add(Activation('sigmoid'))
  model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])
  model.build(dfx_boe.shape)
  # print(model.summary())
  return model

In [13]:
def p_estimation(dfx,dfy):
  X_train, X_test, y_train, y_test = spliter(dfx,dfy)
  for score in scores:
      print("# Tuning hyper-parameters for %s" % score)
      print()
        
      lstm = GridSearchCV(
          KerasClassifier(build_fn=create_model), tuned_parameters, scoring='%s_macro' % score
      )
      lstm.fit(X_train, y_train)

      print("Best parameters set found on development set:")
      print()
      print(lstm.best_params_)
      print()
      print("Grid scores on development set:")
      print()
      means = lstm.cv_results_['mean_test_score']
      stds = lstm.cv_results_['std_test_score']
      for mean, std, params in zip(means, stds, lstm.cv_results_['params']):
          print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params))
      print()

      print("Detailed classification report:")
      print()
      print("The model is trained on the full development set.")
      print("The scores are computed on the full evaluation set.")
      print()
      y_true, y_pred = y_test, lstm.predict(X_test)
      print(classification_report(y_true, y_pred))
      print()

In [ ]:
p_estimation(dfx_boe, dfy_boe)

# Tuning hyper-parameters for precision



In [ ]:
p_estimation(dfx_mean, dfy_mean)

In [ ]:
p_estimation(dfx_median, dfy_median)

In [ ]:
p_estimation(dfx_mostf, dfy_mostf)